In [32]:
from scipy.stats import logser   # SciPy ≥ 1.8


gamma = 0.003

MAX_TRIALS = 200

p = 1 - gamma
k = logser.rvs(p)

if k > MAX_TRIALS:
    print(f"Sampled n_trials {k} exceeds MAX_TRIALS {MAX_TRIALS}. Clamping to MAX_TRIALS.")
    k = MAX_TRIALS
elif k < 1:
    raise ValueError(f"Sampled n_trials {k} is less than 1, which is invalid. Adjust gamma or MAX_TRIALS.")

k

76

In [33]:
import math
import numpy as np

def calculate_overall_dp_from_poisson_per_trial(eps0, delta0, mu, target_epsilon_total):
    """
    Calculates the minimum overall delta_total achievable at a given target_epsilon_total,
    when the base mechanism has (eps0, delta0)-DP and is run K ~ Poisson(mu) times.

    Based on Corollary 21 (Poisson to approx RDP) and Lemma 14 (approx RDP to approx DP)
    from the provided sources.

    Args:
        eps0: Per-trial epsilon for the base mechanism.
        delta0: Per-trial delta for the base mechanism.
        mu: Mean of the Poisson distribution for the number of repetitions K.
        target_epsilon_total: The overall epsilon for which to calculate the minimum delta.

    Returns:
        The calculated minimum overall delta_total for the target_epsilon_total,
        or float('inf') if calculation is not feasible or stable.
    """
    if mu <= 0:
        # Poisson mean must be positive [6]
        return float('inf')
    if eps0 < 0 or delta0 < 0:
        return float('inf')
    if target_epsilon_total < 0:
         return float('inf')

    # Calculate approximate RDP parameters from Corollary 21 [2]
    # delta_prime is the approximate term in approximate RDP
    delta_prime_rdp = 1 - math.exp(-mu * delta0)
    # epsilon_prime is the RDP parameter for the non-approximate part
    # Note: This epsilon_prime is valid for lambda <= 1 + 1/(e^eps0 - 1) per Corollary 21 [2]
    try:
        epsilon_prime_base = eps0 + (math.exp(eps0) - 1) * math.log(mu)
    except ValueError:
         # math.log(mu) fails if mu <= 0, already handled above, but added for safety
         return float('inf')

    # Determine the maximum lambda for which the epsilon_prime_base formula is stated to hold [2, 4]
    try:
        if math.exp(eps0) - 1 > 0:
             max_lambda_for_epsilon_prime = 1 + 1 / (math.exp(eps0) - 1)
        else: # Handles eps0 very close to 0 or 0
             max_lambda_for_epsilon_prime = float('inf') # In limit as eps0 -> 0, 1/(e^eps0-1) -> 1/eps0 -> inf
    except OverflowError: # math.exp(eps0) is too large
         max_lambda_for_epsilon_prime = 1.0 # effectively no lambda > 1 is valid

    # Range of lambda values to check for conversion to (epsilon, delta)-DP [3, 5]
    # We should check lambdas > 1 and within the range where epsilon_prime_base is valid.
    # Choosing a few points for demonstration. A more robust implementation
    # might search this range more thoroughly.
    lambdas_to_check = np.linspace(1.1, min(max_lambda_for_epsilon_prime, 100.0), num=20) # Check up to 100 or max_lambda

    min_delta_total = float('inf')

    for lambda_rdp in lambdas_to_check:
        if lambda_rdp <= 1.0:
             # Lemma 14 conversion formula is for lambda > 1 [3]
             continue

        # Use the conversion formula from Lemma 14:
        # delta_total = delta_prime_rdp + exp((lambda-1)(epsilon_prime - epsilon_total)) / (lambda * (1 - 1/lambda)^(lambda-1)) [3]
        try:
            # Numerically stable calculation of (1 - 1/lambda)^(lambda-1)
            log_term = (lambda_rdp - 1) * math.log(1 - 1/lambda_rdp)
            denominator_term = lambda_rdp * math.exp(log_term)

            # Calculate the exponential term from RDP to DP conversion
            exp_term_conversion = (lambda_rdp - 1) * (epsilon_prime_base - target_epsilon_total)

            # Ensure the exponential term doesn't cause overflow/underflow issues
            if exp_term_conversion > 700: # Approx log(max_float)
                 delta_from_conversion = float('inf')
            elif exp_term_conversion < -700: # Approx log(min_float > 0)
                 delta_from_conversion = 0
            else:
                 delta_from_conversion = math.exp(exp_term_conversion) / denominator_term

            current_delta_total = delta_prime_rdp + delta_from_conversion

            min_delta_total = min(min_delta_total, current_delta_total)

        except (ValueError, OverflowError, ZeroDivisionError) as e:
             # Handle cases where math.log, math.exp, or division by zero might fail
             # print(f"Warning: Calculation failed for lambda={lambda_rdp}: {e}")
             continue # Skip this lambda value

    return min_delta_total

In [52]:
eps0_per_trial = 0.5
delta0_per_trial = 1e-5
mu_poisson = 5.0

target_overall_epsilon = 5.0

overall_delta_calculated = calculate_overall_dp_from_poisson_per_trial(
    eps0_per_trial, delta0_per_trial, mu_poisson, target_overall_epsilon
)

overall_delta_calculated

0.0041807724736178135